In [1]:
import sys
!{sys.executable} -m pip install haystack-ai
!{sys.executable} -m pip install elasticsearch-haystack 
!{sys.executable} -m pip elasticsearch-haystack
!{sys.executable} -m pip install huggingface_hub
!{sys.executable} -m pip  install "huggingface_hub>=0.27.0"
!{sys.executable} -m pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: unknown command "elasticsearch-haystack"
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
from haystack import Pipeline
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchBM25Retriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack.components.preprocessors import DocumentSplitter
from haystack.utils import Secret
from elasticsearch import Elasticsearch, NotFoundError
from glob import glob
import huggingface_hub
import streamlit as st

In [ ]:
# Q3.1: Load Class Transcripts into Elasticsearch

In [3]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip -o transcripts.zip -d transcripts/

--2025-05-07 21:03:36--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-07 21:03:37--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
185.199.108.133, 185.199.109.133, 185.199.110.133, ...tent.com)... 
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
200 OKequest sent, awaiting response... 
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip.6’

transcripts.zip.6   100%[===================>] 284.11K  --.-KB/s    in 0.03s   

2025-05-07 21:03:37 (8.05 MB/s) - ‘transcripts.zip.6’ saved [290933/290933]

Archive:  transcripts.zi

In [4]:
elastic_pwd = "1fVO6RuC"
client = Elasticsearch(hosts="http://localhost:9200", basic_auth=("elastic", elastic_pwd))

index_name = "cs639_lectures"

try:
    client.indices.delete(index=index_name)
except NotFoundError:
    print("Index doesn't exist!")

client.indices.create(index=index_name)

document_store = ElasticsearchDocumentStore(
    hosts="http://localhost:9200",
    basic_auth=("elastic", elastic_pwd),
    index=index_name
)


In [5]:
text_file_paths = [f for f in glob("transcripts/transcripts/*.txt") if not f.endswith(".DS_Store")]
text_file_paths.sort()

In [6]:
index_pipeline = Pipeline()
index_pipeline.add_component("converter", TextFileToDocument())
index_pipeline.add_component("splitter", DocumentSplitter(split_by="word", split_length=150))
index_pipeline.add_component("writer", DocumentWriter(document_store))

index_pipeline.connect("converter", "splitter")
index_pipeline.connect("splitter", "writer")

index_pipeline.run({"converter": {"sources": text_file_paths}})


{'writer': {'documents_written': 901}}

In [7]:
template = """
<|system|>
You are a helpful teaching assistant for the CS 639 Data Management for Data Science course at UW–Madison.<|end|>
<|user|>
Given the following lecture materials, answer the question clearly and concisely.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}<|end|>
<|assistant|>
"""

In [8]:
with open("huggingface_token", "r") as FH:
    data = FH.read()
    huggingface_token = data.strip()

In [9]:
from huggingface_hub import login
login(token=huggingface_token)

In [10]:
retriever = ElasticsearchBM25Retriever(document_store=document_store)
generator = HuggingFaceAPIGenerator(
    api_type="serverless_inference_api",
    api_params={"model": "microsoft/Phi-3.5-mini-instruct", "max_new_tokens": 300 })

In [11]:
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipeline.add_component("llm", generator)

rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


🚅 Components
  - retriever: ElasticsearchBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [12]:
def generate(query):
    assert rag_pipeline is not None
    res = rag_pipeline.run({
        "prompt_builder": {"query": query},
        "retriever": {"query": query}
    })
    return res['llm']['replies'][0]

In [13]:
queries = [
    "Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?"
]

for query in queries:
    response = generate(query)
    top_docs = retriever.run(query=query)["documents"][:3]
    print(f"Query: {query}\nResponse: {response}\n")
    print(f"Top 3 Documents: {top_docs}\n")

/home/SujayCS/.local/lib/python3.10/site-packages/huggingface_hub/inference/_client.py:2288: FutureWarning: `stop_sequences` is a deprecated argument for `text_generation` task and will be removed in version '0.28.0'. Use `stop` instead.
  warnings.warn(


Query: Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?
Response: 1. Setting up GCP VM: Focuses on Google Cloud Platform, specifically creating a small instance.
2. Data storage types: Explores RDBMS, NoSQL DBMS, OODBMS, and graph databases.
3. Data management systems: Introduces AWS Glue and Apache Atlas for cataloging and managing data.
4. Basic Linux and Docker commands: Covers essential Linux commands and Docker environment setup for programming.
5. Data organization tools: Teaches various tools for data storage and management.
6. Data analysis and storytelling: Involves predictive analysis, data dashboards, and data storytelling techniques.

(Note: The provided context does not list six specific projects but rather outlines topics and tools relevant to the course CS 639.)

Top 3 Documents: [Document(id=a28c61bbdb2e5b935cfa3f663d0118e87209af5a103bc017b0cd5c7064b336e7, content: '

In [ ]:
# Q3.2: Implement the Streamlit App

In [14]:
%%writefile app.py

import os
from haystack import Pipeline
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchBM25Retriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter
from haystack.components.preprocessors import DocumentSplitter
from haystack.utils import Secret
from elasticsearch import Elasticsearch, NotFoundError
from glob import glob
import huggingface_hub
import streamlit as st

elastic_pwd = "1fVO6RuC"
client = Elasticsearch(hosts="http://localhost:9200", basic_auth=("elastic", elastic_pwd))

index_name = "cs639_lectures"

try:
    client.indices.delete(index=index_name)
except NotFoundError:
    print("Index doesn't exist!")

client.indices.create(index=index_name)

document_store = ElasticsearchDocumentStore(
    hosts="http://localhost:9200",
    basic_auth=("elastic", elastic_pwd),
    index=index_name
)

text_file_paths = [f for f in glob("transcripts/transcripts/*.txt") if not f.endswith(".DS_Store")]
text_file_paths.sort()

index_pipeline = Pipeline()
index_pipeline.add_component("converter", TextFileToDocument())
index_pipeline.add_component("splitter", DocumentSplitter(split_by="word", split_length=150))
index_pipeline.add_component("writer", DocumentWriter(document_store))

index_pipeline.connect("converter", "splitter")
index_pipeline.connect("splitter", "writer")

index_pipeline.run({"converter": {"sources": text_file_paths}})

template = """
<|system|>
You are a helpful teaching assistant for the CS 639 Data Management for Data Science course at UW–Madison.<|end|>
<|user|>
Given the following lecture materials, answer the question clearly and concisely.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}<|end|>
<|assistant|>
"""

with open("huggingface_token", "r") as FH:
    data = FH.read()
    huggingface_token = data.strip()

from huggingface_hub import login
login(token=huggingface_token)

retriever = ElasticsearchBM25Retriever(document_store=document_store)
generator = HuggingFaceAPIGenerator(
    api_type="serverless_inference_api",
    api_params={"model": "microsoft/Phi-3.5-mini-instruct", "max_new_tokens": 300 })

rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipeline.add_component("llm", generator)

rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

def generate(query):
    assert rag_pipeline is not None
    res = rag_pipeline.run({
        "prompt_builder": {"query": query},
        "retriever": {"query": query}
    })
    print(res.keys())
    return res['llm']['replies'][0]

queries = [
    "Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?",
    "What are some of the topics covered in the course?"
]

for query in queries:
    response = generate(query)
    top_docs = retriever.run(query=query)["documents"][:3]
    print(f"Query: {query}\nResponse: {response}\n")
    print(f"Top 3 Documents: {top_docs}\n")

st.set_page_config(page_title="💬 Course Chatbot", page_icon="📘")
st.title("💬 Course Chatbot")
st.caption("🚀 Interactive Q&A with Elasticsearch, Haystack, and HuggingFace")

if "messages" not in st.session_state:
    st.session_state.messages = []

# Show chat history
for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

# Handle new user input
if prompt := st.chat_input("Ask a question about the course transcripts"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    with st.spinner("Thinking..."):
        response = generate(prompt)

    st.session_state.messages.append({"role": "assistant", "content": response})
    st.chat_message("assistant").write(response)

Overwriting app.py


In [ ]:
# Q3.3: Compare Fine-Tuning vs RAG

In [15]:
queries = [
    "What are some of the topics covered in the course?",
    "What is the course code number for Data Management for Data Science?",
    "What NoSQL database was the main focus of project P2 in CS639?",
    "Can you explain window functions as taught in this course?",
    "What SQL query clauses did we learn about in this course?"
]

for query in queries:
    response = generate(query)
    # top_docs = retriever.run(query=query)["documents"][:3]
    print(f"Query: {query}\n Response: {response}\n")
    # print(f"Top 3 Documents: {top_docs}\n")

Query: What are some of the topics covered in the course?
 Response: The course covers a variety of topics including basic Linux commands, Docker commands, relational database management, SQL, NoSQL with a focus on MongoDB, data cataloging systems like AWS Glue and Apache Atlas, data warehouses, data lakes, semantic layer, knowledge graphs, Apache Iceberg system, ETL vs ELT, data pipelines, predictive analysis using boosting and time series techniques, vector databases, advanced visualization topics like sunburst plots, dashboards, data storytelling, and prompt engineering.

Query: What is the course code number for Data Management for Data Science?
 Response: The course code number for Data Management for Data Science is CS 639.

Query: What NoSQL database was the main focus of project P2 in CS639?
 Response: The main focus of Project P2 in CS639 was MySQL, which is a type of relational database management system (RDBMS).

Query: Can you explain window functions as taught in this cour

In [16]:
# Which approach gave more accurate responses?

"""
The RAG approach appears to be giving more accurate responses in general. 
However, this is not always the case as seen by question 3 where the fine-tuned model managed to identify that P2 was based on  
MongoDB but the RAG model failed to identify this.
However, the responses given to questions 1,4,5 are much better when using the RAG approach comapred to the fine-tuned model.
Both approaches answered question 2 correctly.
"""

# Did the fine-tuned model hallucinate information?
"""
Yes. In Questions 1, 4, and 5, the fine-tuned model produced hallucinated or generic content that did not reflect CS639 accurately.
It often defaulted to common CS course topics instead of what was actually taught.
"""

# Was RAG better at answering new/unseen questions?
"""
Yes, the RAG model performed much better on new or unseen queries (e.g., Questions 4 and 5).
It was able to retrieve relevant documents from the transcripts and generate answers aligned with actual course material.
"""

'\nYes, the RAG model performed much better on new or unseen queries (e.g., Questions 4 and 5).\nIt was able to retrieve relevant documents from the transcripts and generate answers aligned with actual course material.\n'